In [ ]:
#This cell may take a little time to run... be patient!
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from klampt import *
from klampt.vis.ipython import KlamptWidget,Playback
from klampt.math import se3,so3,vectorops
import math

#set up the world with a robot and an environment model
world = WorldModel()
cabinets = world.makeTerrain('cabinets')
cabinets.geometry().loadFile('data/kitchen/MAB 1.obj')
cabinets.geometry().transform(so3.rotation([1,0,0],math.pi/2),[0,0,0])
cabinets.geometry().scale(0.01)
cabinets.geometry().transform(so3.identity(),[1.56,4.49,1.365])
cabinets.appearance().setColor(0.75,0.75,0.75)
table = world.makeTerrain('table')
table.geometry().loadFile('data/kitchen/Dining Table v1.obj')
table.geometry().scale(0.025)
table.geometry().transform(so3.identity(),[1.0,-0.6,0.85])
table.appearance().setColor(0.5,0.3,0.1)
robot = world.makeRigidObject('robot')
geom = robot.geometry()
geom.loadFile('data/Segway_One_Piece.stl')
geom.scale(0.05)
geom.transform(so3.identity(),[0.45,0.6,0])
robot.appearance().setColor(1,0.5,0)

qstart = (0,0,0)
qgoal = (3.2,1.3)

#set up the obstacles list
obstacles = [cabinets.geometry(),table.geometry()]

In [ ]:
#pop up the visualization
kvis=KlamptWidget(world)
kvis.addSphere('start',qstart[0],qstart[1],1.0,r=0.05)
kvis.setColor('start',0,1,0)
kvis.addSphere('goal',qgoal[0],qgoal[1],1.0,r=0.05)
kvis.setColor('goal',1,0,0)

playback_widget = Playback(kvis)
display(kvis)
display(playback_widget)

In [ ]:
from grid import grid_graph
from graph import AdjListGraph
from search import dijkstras,astar_implicit,predecessor_traverse

def sign(x,eps=0):
    return 1 if x > eps else (-1 if x < -eps else 0)

def generate_plan(qstart,qgoal,xrange,yrange,resolution,thetadivs=8,edgeCheckDivs=10):
    #TODO: generate the plan by checking configurations and edges for collision.
    #Don't worry about connecting exactly to the start and goal configurations,
    #you may plan from/to the closest nodes in your grid.
    global robot,obstacles
    xdivs = int((xrange[1]-xrange[0])/resolution)
    ydivs = int((yrange[1]-yrange[0])/resolution)
    delta_theta = math.pi*2/thetadivs
    assert thetadivs == 4 or thetadivs == 8,"Can only do 4-connected or 8-connected grid search"
    #helper functions
    def node_to_config(n):
        if len(n) == 2:
            return (xrange[0] + n[0]*resolution,yrange[0] + n[1]*resolution)
        else:
            return (xrange[0] + n[0]*resolution,yrange[0] + n[1]*resolution,n[2]*delta_theta)
    def config_to_node(q):
        if len(q) == 2:
            return (int((q[0]-xrange[0])/resolution),int((q[1]-yrange[0])/resolution))
        else:
            return (int((q[0]-xrange[0])/resolution),int((q[1]-yrange[0])/resolution),int(q[2]*delta_theta))
    
    #build SE(2) graph
    nodes = []
    edges = []
    for k in range(thetadivs):
        theta = k*delta_theta
        c,s = math.cos(theta),math.sin(theta)
        delta = sign(c,1e-4),sign(s,1e-4)
        G = grid_graph(xdivs,ydivs,diagonals=True)
        for n in G.nodes():
            #add to graph
            nodes.append((n[0],n[1],k))
    print(len(nodes),"nodes passed collision check")
    nodeSet = set(nodes)
    for n in nodes:
        theta = n[2]*delta_theta
        c,s = math.cos(theta),sign(math.sin(theta))
        delta = sign(c,1e-4),sign(s,1e-4)
        nfwd = (n[0]+delta[0],n[1]+delta[1],n[2])
        nbwd = (n[0]-delta[0],n[1]-delta[1],n[2])
        nleft = (n[0],n[1],(n[2]+1)%thetadivs)
        nright = (n[0],n[1],(n[2]-1)%thetadivs)
        if nfwd in nodeSet:
            edges.append((n,nfwd))
        if nbwd in nodeSet:
            edges.append((n,nbwd))
        if nleft in nodeSet:
            edges.append((n,nleft))
        if nright in nodeSet:
            edges.append((n,nright))
    print(len(edges),"edges passed collision check")
    
    G = AdjListGraph(nodes,edges)
    nstart = config_to_node(qstart)
    ngoal = config_to_node(qgoal)
    if nstart not in nodeSet:
        print("START NODE NOT IN GRID")
        return None
    if len(ngoal) == 2:
        xygoal = ngoal
        ngoal = lambda n:n[0]==xygoal[0] and n[1]==xygoal[1]
    path,distances,predecessors = dijkstras(G,nstart,ngoal)
    if path is None:
        print("NO PATH FOUND")
        dmax = 0
        nmax = None
        for n in G.nodes():
            if not math.isinf(distances[n]) and distances[n] > dmax:
                dmax = distances[n]
                nmax = n
        path = predecessor_traverse(predecessors,nstart,nmax)
        print("Returning best path",path)
        return [node_to_config(n) for n in path]
    else:
        return [node_to_config(n) for n in path]

vis_plan_index = 0
def visualize_plan(plan):
    global kvis,playback_widget,robot,qstart,qgoal,vis_plan_index
    kvis.clearExtras()
    vis_plan_index = 0
    def advance(plan=plan):
        global vis_plan_index
        vis_plan_index += 1
        if vis_plan_index >= len(plan):
            vis_plan_index = len(plan)-1
        q = plan[vis_plan_index]
        robot.setTransform(so3.rotation([0,0,1],q[2]),[q[0],q[1],0])
    def reset():
        global vis_plan_index
        vis_plan_index=0
        robot.setTransform(so3.rotation([0,0,1],qstart[2]),[qstart[0],qstart[1],0])
    playback_widget.advance = advance
    playback_widget.reset = reset
    playback_widget.framerate = 1.0
    playback_widget.maxframes = len(plan)
    kvis.addPolyline('plan',[[q[0],q[1],q[2]*0.2] for q in plan])
    
plan = generate_plan(qstart,qgoal,[0,5],[0,3],0.1)
print(plan)
visualize_plan(plan)

In [ ]:
import dd

dd_wheel_base = 0.4
dd_wheel_radius = 0.2

def dd_path_follow(q_path):
    """TODO: put your code here.  Return a sequence of triples (vl,vr,dt)
    that would drive the robot alone the given path"""
    #this drives along an arc for 2 seconds, then spins in place for 4 seconds
    return [(3,0,2),(-3,3,4)]

#No need to change the code below
dd_path = dd_path_follow(plan)
print("Result from dd_path_follow:",dd_path)

dd_dynamics = dd.DifferentialDrive(dd_wheel_base,dd_wheel_radius)
dd_t = 0
dd_q = qstart
animation_dt = 1.0/30.0

def eval_dd_path(dd_path,t):
    for x in dd_path:
        if t < x[2]:
            return x[0],x[1]
        t -= x[2]
    return 0,0

def dd_advance():
    global dd_t,dd_q,dd_path,animation_dt
    dd_t += animation_dt
    dd_q = dd_dynamics.nextState(dd_q,eval_dd_path(dd_path,dd_t),animation_dt)
    robot.setTransform(so3.rotation([0,0,1],dd_q[2]),[dd_q[0],dd_q[1],0])
    
def dd_reset():
    global dd_t,dd_q,qstart
    dd_t = 0
    dd_q = qstart
    robot.setTransform(so3.rotation([0,0,1],dd_q[2]),[dd_q[0],dd_q[1],0])
    
playback_widget.advance = dd_advance
playback_widget.reset = dd_reset
playback_widget.framerate = 30.0
playback_widget.maxframes = sum(x[2] for x in dd_path)*30
